In [ ]:
import os
from openai import OpenAI
from sqlalchemy import create_engine, text

# Database Setup
SERVER = '(localdb)\\MSSQLLocalDB'
DATABASE = 'GroceryDB'
DRIVER = 'ODBC Driver 17 for SQL Server'
DATABASE_URL = f"mssql+pyodbc://{SERVER}/{DATABASE}?driver={DRIVER.replace(' ', '+')}&trusted_connection=yes"
engine = create_engine(DATABASE_URL)

# OpenAI Setup
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def get_recipe_ingredients(recipe_id: str) -> list[dict]:
    """Fetch all ingredients for a recipe from DB."""
    with engine.connect() as conn:
        query = text("""
            SELECT i.IngredientId, i.Name
            FROM RecipeIngredients ri
            JOIN Ingredients i ON ri.IngredientId = i.IngredientId
            WHERE ri.RecipeId = :recipe_id
        """)
        result = conn.execute(query, {"recipe_id": recipe_id})
        return [{"id": row[0], "name": row[1]} for row in result]

def get_major_ingredients(recipe_name: str, ingredients: list[str]) -> list[str]:
    """Ask ChatGPT to identify major ingredients from the list."""
    response = client.chat.completions.create(
        model="gpt-4-turbo",
        messages=[{
            "role": "user",
            "content": f"""
                From this list of ingredients, select the major ingredients for the recipe "{recipe_name}". Return the names in lowercase singular form as a bulleted list.
                the ingredient name should be exactly the same as in the provided ingredient list.
                
                Ingredients:
                {', '.join(ingredients)}
            """
        }],
        temperature=0.1
    )
    ingredients_text = response.choices[0].message.content.strip()
    return [line.replace("-", "").strip() for line in ingredients_text.split("\n") if line.strip()]

def process_recipes():
    """Main workflow: process recipes and save major ingredients."""
    with engine.connect() as conn:
        # Get unprocessed recipes (limit 5 for testing)
        recipes = conn.execute(text("""
            SELECT r.RecipeId, r.Name
            FROM Recipes r
            WHERE NOT EXISTS (
                SELECT 1 FROM MajorIngredientFromRecipeName m 
                WHERE m.RecipeId = r.RecipeId
            )
            --AND r.RecipeId = '10014'
        """)).fetchall()

        for recipe_id, recipe_name in recipes:
            try:
                # Get all ingredients for this recipe
                ingredients = get_recipe_ingredients(recipe_id)
                ingredient_names = [i["name"] for i in ingredients]
                
                # Get major ingredients from ChatGPT
                major_ingredients = get_major_ingredients(recipe_name, ingredient_names)
                print(f"Recipe: {recipe_name}\nMajor ingredients: {major_ingredients}")
                
                # Save to database
                for ingredient in ingredients:
                    if ingredient["name"].lower() in major_ingredients:
                        conn.execute(text("""
                            INSERT INTO MajorIngredientFromRecipeName (RecipeId, IngredientId, LLM)
                            VALUES (:recipe_id, :ingredient_id, 'GPT-4')
                        """), {
                            "recipe_id": recipe_id,
                            "ingredient_id": ingredient["id"]
                        })
                conn.commit()
                
            except Exception as e:
                print(f"Error processing {recipe_name}: {e}")
                conn.rollback()

if __name__ == "__main__":
    process_recipes()
   
    

Recipe: Brussels Sprouts With Walnuts & Arugula
Major ingredients: ['brussels sprouts', 'walnut pieces', 'arugula']
Recipe: Red Leicester Cheese, Lentils And Red Cabbage
Major ingredients: ['red leicester cheese', 'lentils', 'red cabbage']
Recipe: Cabbage And Lime Salad With Roasted Peanuts
Major ingredients: ['cabbage', 'lime', 'roasted peanuts']
Recipe: Mushrooms, Cannellini Beans And Truffle Cream
Major ingredients: ['cannellini bean', 'crimini mushroom']
Recipe: Dandelion Greens with Currants and Pine Nuts
Major ingredients: ['dandelion green', 'pine nut', 'dried currant']
Recipe: What’s So Great About Kale
Major ingredients: ['It seems like the list of ingredients was not provided in your message. Could you please provide the list of ingredients so I can help you identify the major ingredients for the recipe "What’s So Great About Kale"?']
Recipe: Peas & Pancetta
Major ingredients: ['pea', 'pancetta']
Recipe: White Mushrooms, Peas And Mixed Leaves
Major ingredients: ['fresh mushro